In [2]:
import numpy as np
import pandas as pd

## 读入数据，并对2基站情况数据进行合并与拆解

In [3]:
resource=pd.read_csv('RAWDATA/test_set_2_2ap.csv')

   #删掉异常的行值

In [4]:
resource.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64 entries, 0 to 63
Data columns (total 46 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   test_id                      64 non-null     int64  
 1   test_dur                     64 non-null     int64  
 2   loc_id                       64 non-null     object 
 3   protocol                     64 non-null     object 
 4   pkt_len                      64 non-null     int64  
 5   bss_id                       64 non-null     int64  
 6   ap_name                      64 non-null     object 
 7   ap_mac                       64 non-null     object 
 8   ap_id                        64 non-null     object 
 9   pd                           64 non-null     int64  
 10  ed                           64 non-null     int64  
 11  nav                          64 non-null     int64  
 12  eirp                         64 non-null     int64  
 13  ap_from_ap_0_sum_ant_r

In [5]:
columns=[0,1,2,4,6,7,8,12,19,20,22,23,25,26,28,29,31,32,35,36,37,38,39,40,42,43,44,45]
resource=resource.drop(resource.columns[columns],axis=1)

In [6]:
resource.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64 entries, 0 to 63
Data columns (total 18 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   protocol                    64 non-null     object 
 1   bss_id                      64 non-null     int64  
 2   pd                          64 non-null     int64  
 3   ed                          64 non-null     int64  
 4   nav                         64 non-null     int64  
 5   ap_from_ap_0_sum_ant_rssi   32 non-null     object 
 6   ap_from_ap_0_max_ant_rssi   32 non-null     object 
 7   ap_from_ap_0_mean_ant_rssi  32 non-null     object 
 8   ap_from_ap_1_sum_ant_rssi   32 non-null     object 
 9   ap_from_ap_1_max_ant_rssi   32 non-null     object 
 10  ap_from_ap_1_mean_ant_rssi  32 non-null     object 
 11  sta_to_ap_0_sum_ant_rssi    64 non-null     object 
 12  sta_to_ap_1_sum_ant_rssi    64 non-null     object 
 13  sta_from_ap_0_sum_ant_rssi  64 non-nu

In [7]:
# 合并操作
for i in range(resource.shape[0]):
    if resource.iloc[i,list(resource.columns).index('bss_id')]==0:
        resource.iloc[i,list(resource.columns).index('ap_from_ap_0_sum_ant_rssi')]=resource.iloc[i,list(resource.columns).index('ap_from_ap_1_sum_ant_rssi')]
        resource.iloc[i,list(resource.columns).index('ap_from_ap_0_max_ant_rssi')]=resource.iloc[i,list(resource.columns).index('ap_from_ap_1_max_ant_rssi')]
        resource.iloc[i,list(resource.columns).index('ap_from_ap_0_mean_ant_rssi')]=resource.iloc[i,list(resource.columns).index('ap_from_ap_1_mean_ant_rssi')]
        resource.iloc[i,list(resource.columns).index('sta_from_sta_0_rssi')]=resource.iloc[i,list(resource.columns).index('sta_from_sta_1_rssi')]

In [8]:
resource=resource.drop(['ap_from_ap_1_sum_ant_rssi','ap_from_ap_1_max_ant_rssi',
                       'ap_from_ap_1_mean_ant_rssi','sta_from_sta_1_rssi'],axis=1)

In [9]:
resource.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64 entries, 0 to 63
Data columns (total 14 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   protocol                    64 non-null     object 
 1   bss_id                      64 non-null     int64  
 2   pd                          64 non-null     int64  
 3   ed                          64 non-null     int64  
 4   nav                         64 non-null     int64  
 5   ap_from_ap_0_sum_ant_rssi   64 non-null     object 
 6   ap_from_ap_0_max_ant_rssi   64 non-null     object 
 7   ap_from_ap_0_mean_ant_rssi  64 non-null     object 
 8   sta_to_ap_0_sum_ant_rssi    64 non-null     object 
 9   sta_to_ap_1_sum_ant_rssi    64 non-null     object 
 10  sta_from_ap_0_sum_ant_rssi  64 non-null     object 
 11  sta_from_ap_1_sum_ant_rssi  64 non-null     object 
 12  sta_from_sta_0_rssi         64 non-null     float64
 13  seq_time                    0 non-nul

In [10]:
# 定义管理rssi中异常值数据，并创造出新特征值
def cal_pd(apmax,pd):
    apmax = eval(apmax)
    comparison_result = [1 if x >= pd else 0 for x in apmax]
    proportion_of_ones = sum(comparison_result) / len(apmax)
    return proportion_of_ones

def cal_ed(apmax,ed):
    apmax = eval(apmax)
    comparison_result = [1 if x >= ed else 0 for x in apmax]
    proportion_of_ones = sum(comparison_result) / len(apmax)
    return proportion_of_ones

def cal_nav(apmean,nav):
    apmean = eval(apmean)
    comparison_result = [1 if x >= nav else 0 for x in apmean]

    # 计算 1 的占比
    proportion_of_ones = sum(comparison_result) / len(apmean)
    return proportion_of_ones    

In [11]:
resource['pro_pd'] = resource.apply(lambda row: cal_pd(row['ap_from_ap_0_max_ant_rssi'], row['pd']), axis=1)
resource['pro_ed'] = resource.apply(lambda row: cal_ed(row['ap_from_ap_0_max_ant_rssi'], row['ed']), axis=1)
resource['pro_nav'] = resource.apply(lambda row: cal_nav(row['ap_from_ap_0_mean_ant_rssi'], row['nav']), axis=1)

In [12]:
drop_columns=['pd','ed','ap_from_ap_0_mean_ant_rssi','ap_from_ap_0_max_ant_rssi']

# drop_columns=['pd','ed','ap_from_ap_0_mean_ant_rssi','ap_from_ap_0_max_ant_rssi'
#               ,'sta_to_ap_0_max_ant_rssi','sta_to_ap_0_mean_ant_rssi'
#               ,'sta_to_ap_1_max_ant_rssi','sta_to_ap_1_mean_ant_rssi'
#              ,'sta_from_ap_0_max_ant_rssi','sta_from_ap_0_mean_ant_rssi'
#              ,'sta_from_ap_1_max_ant_rssi','sta_from_ap_1_mean_ant_rssi']

resource=resource.drop(drop_columns,axis=1)

In [13]:
resource.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64 entries, 0 to 63
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   protocol                    64 non-null     object 
 1   bss_id                      64 non-null     int64  
 2   nav                         64 non-null     int64  
 3   ap_from_ap_0_sum_ant_rssi   64 non-null     object 
 4   sta_to_ap_0_sum_ant_rssi    64 non-null     object 
 5   sta_to_ap_1_sum_ant_rssi    64 non-null     object 
 6   sta_from_ap_0_sum_ant_rssi  64 non-null     object 
 7   sta_from_ap_1_sum_ant_rssi  64 non-null     object 
 8   sta_from_sta_0_rssi         64 non-null     float64
 9   seq_time                    0 non-null      float64
 10  pro_pd                      64 non-null     float64
 11  pro_ed                      64 non-null     float64
 12  pro_nav                     64 non-null     float64
dtypes: float64(5), int64(2), object(6)
me

In [14]:
# 对RSSI求平均值处理
for i in range(3,8):
    resource.iloc[:,i]=resource.iloc[:,i].apply(lambda x: sum(eval(x))/len(eval(x)))

In [15]:
# 选择pro的聚类个数
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt

data_scaled=resource.iloc[:,[10,11,12]]

df=data_scaled
k = 3
kmeans = KMeans(n_clusters=k, random_state=42)
df['cluster'] = kmeans.fit_predict(data_scaled)

# 查看每个簇的平均特征值
cluster_centers = pd.DataFrame(kmeans.cluster_centers_)
# columns=['pro_pd', 'pro_ed', 'pro_nav']

print(f"Cluster centers:\n{cluster_centers}")

# 查看每个簇的样本数量
print(df['cluster'].value_counts())


D:\Anaconda\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
D:\Anaconda\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


Cluster centers:
          0         1         2
0  0.956025  0.004927  0.984550
1  0.064212  0.004366  0.057344
2  0.180631  0.003470  0.461771
cluster
0    47
1    13
2     4
Name: count, dtype: int64


C:\Users\ShenFei\AppData\Local\Temp\ipykernel_19596\2563942475.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cluster'] = kmeans.fit_predict(data_scaled)


In [16]:
resource['state']=df['cluster']

In [17]:
resource.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64 entries, 0 to 63
Data columns (total 14 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   protocol                    64 non-null     object 
 1   bss_id                      64 non-null     int64  
 2   nav                         64 non-null     int64  
 3   ap_from_ap_0_sum_ant_rssi   64 non-null     object 
 4   sta_to_ap_0_sum_ant_rssi    64 non-null     object 
 5   sta_to_ap_1_sum_ant_rssi    64 non-null     object 
 6   sta_from_ap_0_sum_ant_rssi  64 non-null     object 
 7   sta_from_ap_1_sum_ant_rssi  64 non-null     object 
 8   sta_from_sta_0_rssi         64 non-null     float64
 9   seq_time                    0 non-null      float64
 10  pro_pd                      64 non-null     float64
 11  pro_ed                      64 non-null     float64
 12  pro_nav                     64 non-null     float64
 13  state                       64 non-nu

In [18]:
# 信干比采取dbm直接相减得到db
def func(x, y, state):
    if state == 0:
        return x + 100
    else:
        return x - y

    
# 创建一个新列来存储 SINR 计算的结果
sinr = []
df = resource
# 遍历 DataFrame 的每一行
for i in range(len(df)):
    if df.iloc[i, list(df.columns).index('bss_id')] == 1:  # ap1
        result = func(df.iloc[i, list(df.columns).index('sta_from_ap_1_sum_ant_rssi')], df.iloc[i, list(df.columns).index('sta_from_ap_0_sum_ant_rssi')],df.iloc[i, list(df.columns).index('state')])
    else:  # ap2
        result = func(df.iloc[i, list(df.columns).index('sta_from_ap_0_sum_ant_rssi')], df.iloc[i, list(df.columns).index('sta_from_ap_1_sum_ant_rssi')],df.iloc[i, list(df.columns).index('state')])
    sinr.append(result)

# 将计算结果添加为新列
df['sinr'] = sinr

# 输出结果
print(df['sinr'])
resource = resource.drop('bss_id',axis=1)

0     16.503333
1     24.575000
2     16.503333
3     24.575000
4     14.677632
        ...    
59    54.850000
60    59.875000
61    56.450000
62    59.875000
63    56.450000
Name: sinr, Length: 64, dtype: float64


In [19]:
resource = resource.loc[resource['sinr'] > 0]

In [20]:
resource.iloc[:,0]=resource.iloc[:,0].astype(str).map({'tcp':1,'udp':0})

# read from here

In [28]:
# resource.to_csv('programe1_predict_test_2_type_2.csv')
import numpy as np
import pandas as pd
resource = pd.read_excel('programe1_2ap_processed.xlsx')
resource = resource.sample(frac=1).reset_index(drop=True)

target=resource['seq_time']
resource=resource.drop('seq_time',axis=1)
resource.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 379 entries, 0 to 378
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   protocol                    379 non-null    int64  
 1   nav                         379 non-null    int64  
 2   ap_from_ap_0_sum_ant_rssi   379 non-null    float64
 3   sta_to_ap_0_sum_ant_rssi    379 non-null    float64
 4   sta_to_ap_1_sum_ant_rssi    379 non-null    float64
 5   sta_from_ap_0_sum_ant_rssi  379 non-null    float64
 6   sta_from_ap_1_sum_ant_rssi  379 non-null    float64
 7   sta_from_sta_0_rssi         379 non-null    int64  
 8   pro_pd                      379 non-null    float64
 9   pro_ed                      379 non-null    float64
 10  pro_nav                     379 non-null    float64
 11  state                       379 non-null    int64  
 12  sinr                        379 non-null    float64
dtypes: float64(9), int64(4)
memory usag

In [29]:
from sklearn.ensemble import RandomForestRegressor

rfg=RandomForestRegressor(n_estimators=100,criterion='friedman_mse',random_state=0,oob_score=False)


# target_0_phy=pd.DataFrame(target_0_phy)

rfg=rfg.fit(resource,target)

# rfg0=RandomForestRegressor(n_estimators=100,criterion='friedman_mse',random_state=0,oob_score=False)
# rfg0=rfg0.fit(resource,target_0.to_numpy())
from sklearn.model_selection import cross_val_score

res=cross_val_score(rfg,resource,target,cv=10)

In [30]:
res

array([0.87119611, 0.92985955, 0.89552621, 0.92080221, 0.776792  ,
       0.95553315, 0.82261539, 0.79558939, 0.86823936, 0.91384649])

In [31]:
res.mean()

0.8749999863348752

# XGBoost

In [62]:
from xgboost import XGBRegressor as XGBR
reg=XGBR(n_estimators=100).fit(resource.to_numpy(),target.to_numpy())

In [64]:
res=cross_val_score(reg,resource.to_numpy(),target.to_numpy(),cv=10)

In [65]:
res.mean()

0.834313413362888

In [66]:
res

array([0.917585  , 0.82404495, 0.89327948, 0.92782213, 0.72041921,
       0.89140723, 0.71889142, 0.79739348, 0.95518596, 0.69710528])

# 岭回归

In [67]:
from sklearn.linear_model import Ridge

regcv=Ridge(alpha=1)
regcv=regcv.fit(resource,target)
# regcv.score(resource_merge,target_merge)

res=cross_val_score(regcv,resource,target,cv=10)

In [68]:
res.mean()

0.8519286230798354

In [69]:
res

array([0.87765508, 0.89335851, 0.90123807, 0.82937039, 0.74822159,
       0.82066127, 0.85470823, 0.7702948 , 0.94442736, 0.87935094])

# 构造predict

In [3]:
import numpy as np
import pandas as pd
predict=pd.read_csv('./RAWDATA/test_set_1_2ap.csv')

In [4]:
predict.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80 entries, 0 to 79
Data columns (total 46 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   test_id                      80 non-null     int64  
 1   test_dur                     80 non-null     int64  
 2   loc_id                       80 non-null     object 
 3   protocol                     80 non-null     object 
 4   pkt_len                      80 non-null     int64  
 5   bss_id                       80 non-null     int64  
 6   ap_name                      80 non-null     object 
 7   ap_mac                       80 non-null     object 
 8   ap_id                        80 non-null     object 
 9   pd                           80 non-null     int64  
 10  ed                           80 non-null     int64  
 11  nav                          80 non-null     int64  
 12  eirp                         80 non-null     int64  
 13  ap_from_ap_0_sum_ant_r

In [6]:
columns=[0,1,2,4,6,7,8,12,19,20,22,23,25,26,28,29,31,32,35,36,37,38,39,40,42,43,44,45]
predict=predict.drop(predict.columns[columns],axis=1)

In [7]:
# 合并操作
for i in range(predict.shape[0]):
    if predict.iloc[i,list(predict.columns).index('bss_id')]==0:
        predict.iloc[i,list(predict.columns).index('ap_from_ap_0_sum_ant_rssi')]=predict.iloc[i,list(predict.columns).index('ap_from_ap_1_sum_ant_rssi')]
        predict.iloc[i,list(predict.columns).index('ap_from_ap_0_max_ant_rssi')]=predict.iloc[i,list(predict.columns).index('ap_from_ap_1_max_ant_rssi')]
        predict.iloc[i,list(predict.columns).index('ap_from_ap_0_mean_ant_rssi')]=predict.iloc[i,list(predict.columns).index('ap_from_ap_1_mean_ant_rssi')]
        predict.iloc[i,list(predict.columns).index('sta_from_sta_0_rssi')]=predict.iloc[i,list(predict.columns).index('sta_from_sta_1_rssi')]

In [8]:
predict=predict.drop(['ap_from_ap_1_sum_ant_rssi','ap_from_ap_1_max_ant_rssi',
                       'ap_from_ap_1_mean_ant_rssi','sta_from_sta_1_rssi'],axis=1)

In [9]:
predict.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80 entries, 0 to 79
Data columns (total 14 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   protocol                    80 non-null     object 
 1   bss_id                      80 non-null     int64  
 2   pd                          80 non-null     int64  
 3   ed                          80 non-null     int64  
 4   nav                         80 non-null     int64  
 5   ap_from_ap_0_sum_ant_rssi   80 non-null     object 
 6   ap_from_ap_0_max_ant_rssi   80 non-null     object 
 7   ap_from_ap_0_mean_ant_rssi  80 non-null     object 
 8   sta_to_ap_0_sum_ant_rssi    80 non-null     object 
 9   sta_to_ap_1_sum_ant_rssi    80 non-null     object 
 10  sta_from_ap_0_sum_ant_rssi  80 non-null     object 
 11  sta_from_ap_1_sum_ant_rssi  80 non-null     object 
 12  sta_from_sta_0_rssi         0 non-null      float64
 13  seq_time                    0 non-nul

In [11]:
# 定义管理rssi中异常值数据，并创造出新特征值
def cal_pd(apmax,pd):
    apmax = eval(apmax)
    comparison_result = [1 if x >= pd else 0 for x in apmax]
    proportion_of_ones = sum(comparison_result) / len(apmax)
    return proportion_of_ones

def cal_ed(apmax,ed):
    apmax = eval(apmax)
    comparison_result = [1 if x >= ed else 0 for x in apmax]
    proportion_of_ones = sum(comparison_result) / len(apmax)
    return proportion_of_ones

def cal_nav(apmean,nav):
    apmean = eval(apmean)
    comparison_result = [1 if x >= nav else 0 for x in apmean]

    # 计算 1 的占比
    proportion_of_ones = sum(comparison_result) / len(apmean)
    return proportion_of_ones    

In [12]:
predict['pro_pd'] = predict.apply(lambda row: cal_pd(row['ap_from_ap_0_max_ant_rssi'], row['pd']), axis=1)
predict['pro_ed'] = predict.apply(lambda row: cal_ed(row['ap_from_ap_0_max_ant_rssi'], row['ed']), axis=1)
predict['pro_nav'] = predict.apply(lambda row: cal_nav(row['ap_from_ap_0_mean_ant_rssi'], row['nav']), axis=1)

In [14]:
drop_columns=['pd','ed','ap_from_ap_0_mean_ant_rssi','ap_from_ap_0_max_ant_rssi']

# drop_columns=['pd','ed','ap_from_ap_0_mean_ant_rssi','ap_from_ap_0_max_ant_rssi'
#               ,'sta_to_ap_0_max_ant_rssi','sta_to_ap_0_mean_ant_rssi'
#               ,'sta_to_ap_1_max_ant_rssi','sta_to_ap_1_mean_ant_rssi'
#              ,'sta_from_ap_0_max_ant_rssi','sta_from_ap_0_mean_ant_rssi'
#              ,'sta_from_ap_1_max_ant_rssi','sta_from_ap_1_mean_ant_rssi']

predict=predict.drop(drop_columns,axis=1)

In [17]:
# 对RSSI求平均值处理
for i in range(3,8):
    predict.iloc[:,i]=predict.iloc[:,i].apply(lambda x: sum(eval(x))/len(eval(x)))

In [19]:
# 选择pro的聚类个数
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt

data_scaled=predict.iloc[:,[10,11,12]]

df=data_scaled
k = 3
kmeans = KMeans(n_clusters=k, random_state=42)
df['cluster'] = kmeans.fit_predict(data_scaled)

# 查看每个簇的平均特征值
cluster_centers = pd.DataFrame(kmeans.cluster_centers_)
# columns=['pro_pd', 'pro_ed', 'pro_nav']

print(f"Cluster centers:\n{cluster_centers}")

# 查看每个簇的样本数量
print(df['cluster'].value_counts())


D:\Anaconda\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
D:\Anaconda\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


Cluster centers:
          0             1         2
0  0.053833  2.124077e-03  0.005862
1  0.876570  1.424501e-03  0.893614
2  0.772115 -4.336809e-19  0.107937
cluster
1    36
0    34
2    10
Name: count, dtype: int64


C:\Users\ShenFei\AppData\Local\Temp\ipykernel_18632\3987966281.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cluster'] = kmeans.fit_predict(data_scaled)


In [20]:
predict['state']=df['cluster']

In [21]:
predict.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80 entries, 0 to 79
Data columns (total 14 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   protocol                    80 non-null     object 
 1   bss_id                      80 non-null     int64  
 2   nav                         80 non-null     int64  
 3   ap_from_ap_0_sum_ant_rssi   80 non-null     object 
 4   sta_to_ap_0_sum_ant_rssi    80 non-null     object 
 5   sta_to_ap_1_sum_ant_rssi    80 non-null     object 
 6   sta_from_ap_0_sum_ant_rssi  80 non-null     object 
 7   sta_from_ap_1_sum_ant_rssi  80 non-null     object 
 8   sta_from_sta_0_rssi         0 non-null      float64
 9   seq_time                    0 non-null      float64
 10  pro_pd                      80 non-null     float64
 11  pro_ed                      80 non-null     float64
 12  pro_nav                     80 non-null     float64
 13  state                       80 non-nu

In [22]:
# 信干比采取dbm直接相减得到db
def func(x, y, state):
    if state == 0:
        return x + 100
    else:
        return x - y

    
# 创建一个新列来存储 SINR 计算的结果
sinr = []
df = predict
# 遍历 DataFrame 的每一行
for i in range(len(df)):
    if df.iloc[i, list(df.columns).index('bss_id')] == 1:  # ap1
        result = func(df.iloc[i, list(df.columns).index('sta_from_ap_1_sum_ant_rssi')], df.iloc[i, list(df.columns).index('sta_from_ap_0_sum_ant_rssi')],df.iloc[i, list(df.columns).index('state')])
    else:  # ap2
        result = func(df.iloc[i, list(df.columns).index('sta_from_ap_0_sum_ant_rssi')], df.iloc[i, list(df.columns).index('sta_from_ap_1_sum_ant_rssi')],df.iloc[i, list(df.columns).index('state')])
    sinr.append(result)

# 将计算结果添加为新列
df['sinr'] = sinr

# 输出结果
print(df['sinr'])
predict = predict.drop('bss_id',axis=1)

0     27.700000
1     31.050000
2     27.700000
3     31.050000
4     28.625000
        ...    
75    16.843590
76    31.207143
77    19.727632
78    31.207143
79    19.727632
Name: sinr, Length: 80, dtype: float64


In [23]:
predict.iloc[:,0]=predict.iloc[:,0].astype(str).map({'tcp':1,'udp':0})

In [24]:
# predict.to_csv('programe1_predict_type2.csv')

# predict from here

In [22]:
resource.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 379 entries, 0 to 378
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   protocol                    379 non-null    int64  
 1   nav                         379 non-null    int64  
 2   ap_from_ap_0_sum_ant_rssi   379 non-null    float64
 3   sta_to_ap_0_sum_ant_rssi    379 non-null    float64
 4   sta_to_ap_1_sum_ant_rssi    379 non-null    float64
 5   sta_from_ap_0_sum_ant_rssi  379 non-null    float64
 6   sta_from_ap_1_sum_ant_rssi  379 non-null    float64
 7   sta_from_sta_0_rssi         379 non-null    int64  
 8   pro_pd                      379 non-null    float64
 9   pro_ed                      379 non-null    float64
 10  pro_nav                     379 non-null    float64
 11  state                       379 non-null    int64  
 12  sinr                        379 non-null    float64
dtypes: float64(9), int64(4)
memory usag

In [24]:
predict=pd.read_csv('programe1_predict_test_2_type_2.csv')

In [32]:
predict.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64 entries, 0 to 63
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   protocol                    64 non-null     int64  
 1   nav                         64 non-null     int64  
 2   ap_from_ap_0_sum_ant_rssi   64 non-null     float64
 3   sta_to_ap_0_sum_ant_rssi    64 non-null     float64
 4   sta_to_ap_1_sum_ant_rssi    64 non-null     float64
 5   sta_from_ap_0_sum_ant_rssi  64 non-null     float64
 6   sta_from_ap_1_sum_ant_rssi  64 non-null     float64
 7   sta_from_sta_0_rssi         64 non-null     int64  
 8   pro_pd                      64 non-null     float64
 9   pro_ed                      64 non-null     float64
 10  pro_nav                     64 non-null     float64
 11  state                       64 non-null     int64  
 12  sinr                        64 non-null     float64
dtypes: float64(9), int64(4)
memory usage:

In [26]:
predict=predict.drop('seq_time',axis=1)

In [7]:
resource=resource.drop('sta_from_sta_0_rssi',axis=1)

In [8]:
from sklearn.ensemble import RandomForestRegressor

rfg=RandomForestRegressor(n_estimators=100,criterion='friedman_mse',random_state=0,oob_score=False)


rfg=rfg.fit(resource,target)

from sklearn.model_selection import cross_val_score

res=cross_val_score(rfg,resource,target,cv=10)

In [9]:
res.mean()

0.8722996397546288

In [33]:
res=rfg.predict(predict)

In [34]:
pd.DataFrame(res).to_excel('programe1_test_2_type2_res.xlsx')